In [1]:
% load_ext autoreload
% autoreload 2
import os

if not os.getcwd().endswith('mob2crime'):
    os.chdir('..')
os.getcwd()

'/home/Jiahui/mob2crime'

In [2]:
from src.utils.gis import *
import  src.mex_helper as mex

In [3]:
%matplotlib inline

In [4]:
mex_grids = mex.grids(rkind='metropolitans_16', side=3000)

reading existing grids


In [5]:
mex_grids.groupby('metropolitan').apply(len)

metropolitan
Acapulco de Juárez, Guerrero          478
Cancún, Quintana Roo                  425
Chihuahua, Chihuahua                 2102
Cuernavaca, Morelos                   173
Culiacán Rosales, Sinaloa             791
Guadalajara, Jalisco                  359
Juárez, Chihuahua                     435
Mexicali, Baja California            1856
Monterrey, Nuevo León                 854
Morelia, Michoacán de Ocampo          252
Nuevo Laredo, Tamaulipas              161
Oaxaca de Juárez, Oaxaca              101
Tijuana, Baja California              530
Toluca de Lerdo, México               296
Valle de Mexico, Distrito Federal     208
Villahermosa, Tabasco                 318
dtype: int64

In [7]:

%load_ext line_profiler


In [6]:

polys = mex_grids[mex_grids.metropolitan=='Valle de Mexico, Distrito Federal']

rgrids = polys

import pandas as pd

n_grids = len(rgrids)
sqrt_area = 100
cgrids_avg = pd.DataFrame(np.random.randint(0, 100,size=(n_grids,24)))
s = cgrids_avg / cgrids_avg.sum()
s.index = range(n_grids)



In [8]:
def fast_dv(rgrids, s, sqrt_area):
    grid_dist = polys_centroid_pairwise_dist(rgrids, dist_crs=mex.EQDC_CRS)
    dv = {}
    for t in s.columns:
        st_outer = np.outer(s[t], s[t])
        np.fill_diagonal(st_outer, 0)
        dv[int(t)] = (st_outer * grid_dist).sum() / st_outer.sum() / sqrt_area
    return dv

In [9]:
%lprun -f fast_dv fast_dv(rgrids, s, sqrt_area)


In [10]:
def slow_dv(rgrids, s, sqrt_area):
    dv = {}
    grid_cens = np.array(rgrids.to_crs(mex.EQDC_CRS).geometry.apply(lambda x: x.centroid.coords[0]).tolist())
    for t in s.columns:
        st = s[t].tolist()
        st_sum = 0
        st_grid_dist = 0
        for i in range(n_grids):
            sti = st[i]
            gi = grid_cens[i]
            for j in range(i+1, n_grids):
                gj = grid_cens[j]
                st_ij = sti*st[j]
                st_sum += st_ij
                dist = np.linalg.norm(gi-gj)
                st_grid_dist += st_ij * dist
        dv[t] = st_grid_dist/st_sum/sqrt_area
    return dv 

In [11]:
%lprun -f slow_dv slow_dv(rgrids, s, sqrt_area)

In [12]:
dv1 = fast_dv(rgrids, s, sqrt_area)
dv2 = slow_dv(rgrids, s, sqrt_area)

In [14]:
diff = pd.DataFrame(list(zip(dv1.values(),dv2.values())))
diff[0]/diff[1]

0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     1.0
8     1.0
9     1.0
10    1.0
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    1.0
22    1.0
23    1.0
dtype: float64